In [38]:
import functools
import pprint as pp
import shlex
import os
import inspect
import ast
from subprocess import run

In [40]:
def memory_profile(f):
    @functools.wraps(f)
    def inner(*args, **kwargs):
        
        def unwrap_source_code(g):
            sourcelines = inspect.getsourcelines(g)
            for idx, i in enumerate(sourcelines[0]):
                if i.startswith('def'):
                    return "".join(sourcelines[0][idx:])
            
        temp = './temp_ram.py'
        with open(temp, 'w+') as g:
            decorator = '@profile'
            sourcelines = inspect.getsourcelines(f)
            value = f"""import ast\nimport argparse\nimport sys\nimport json\n\n\n{decorator}\n{unwrap_source_code(f)}"""
            g.write(value)
            g.write('\n')
          
            main_exec = f"""if __name__ == "__main__":     
    parser = argparse.ArgumentParser()
    parser.add_argument('-dummy_pos', nargs='*')
    parser.add_argument('--dummy_optional', nargs="?")    
    args = parser.parse_args()

    star_args = args.dummy_pos
    optional_args = args.dummy_optional

    if optional_args:
        optional_args = optional_args.replace("\'", '"')
        optional_args = json.loads(json.loads(json.dumps(optional_args)))
    else:
        optional_args = {{}} 
     
    dummy_args = []
    for arg in star_args:
        if arg.isnumeric():
            dummy_args.append(ast.literal_eval(arg))
        else:
            dummy_args.append(arg)
        
    
    {f.__name__}(*dummy_args, **optional_args)
                    """
            g.write(main_exec)
            
        cmd = f'python -m memory_profiler {temp}'
        if args:
            args_split = [f'"{str(i)}"' for i in args]
            second_split =  "-dummy_pos" + " " + " ".join(args_split)
           
            cmd = cmd + " " + second_split
        
        if kwargs:
            cmd = cmd + " " + '--dummy_optional' + " " + " ".join([f'"{ {k : str(v) for k, v in kwargs.items()} }"'])
        
        print(cmd)
        mem_stats = run(cmd, capture_output=True).stdout.decode('utf-8')
        
       
        # TODO: We might want to have also 
        # a `debug` argument, where we don't delete
        # the temporary file.
        
        #os.remove(temp)
        print(mem_stats)
        return f(*args, **kwargs)
    return inner
    
    
    
def timing(f):
    import time
    @functools.wraps(f)
    def inner(*args, **kwargs):
        start = time.perf_counter()
        value = f(*args, **kwargs)
        end = time.perf_counter()
        print(f"Finished {f.__name__} in {end - start:.4f} seconds.")
        return value
    return inner


    


In [50]:
@timing
@memory_profile
def my_func(x, y):
    a = [1] * (10 ** x)
    b = [2] * (1 * 10 ** y)
    c = a + b
    del c
    return a



